In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, Dropout, Dense, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize the images
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# One hot encode the labels
num_classes = 10
train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

170498071/170498071 [==============================] - 4s 0us/step


In [2]:
# Data augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(train_images)

In [3]:
# Creating a sequential model and adding layers to it
from tensorflow.keras import layers, models


model = Sequential()

model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))    # num_classes = 10

# Checking the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                        

In [4]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [5]:
# Learning rate scheduler
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 60:
        lr *= 1e-3
    elif epoch > 40:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 1e-1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

In [6]:
from keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
model_checkpoint = ModelCheckpoint(
    filepath='/content/{epoch:02d}.ckpt',  # Filepath to save the model
    save_freq = "epoch",
    period = 10,
    save_weights_only=True,  # Save only the model weights
    verbose=1
)

callbacks_list = [model_checkpoint]
# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=64),
                    #steps_per_epoch=len(train_images) // 64,
                    epochs=100,
                    validation_data=(test_images, test_labels),
                    callbacks=[lr_scheduler, model_checkpoint])

Epoch 1/100
782/782 [==============================] - 55s 51ms/step - loss: 1.8244 - accuracy: 0.3652 - val_loss: 1.4569 - val_accuracy: 0.4886 - lr: 0.0010
Epoch 2/100
782/782 [==============================] - 34s 44ms/step - loss: 1.3306 - accuracy: 0.5210 - val_loss: 1.4138 - val_accuracy: 0.5450 - lr: 0.0010
Epoch 3/100
782/782 [==============================] - 34s 43ms/step - loss: 1.1210 - accuracy: 0.6032 - val_loss: 1.0591 - val_accuracy: 0.6449 - lr: 0.0010
Epoch 4/100
782/782 [==============================] - 34s 44ms/step - loss: 1.0083 - accuracy: 0.6477 - val_loss: 0.9742 - val_accuracy: 0.6584 - lr: 0.0010
Epoch 5/100
782/782 [==============================] - 36s 46ms/step - loss: 0.9281 - accuracy: 0.6754 - val_loss: 1.0137 - val_accuracy: 0.6666 - lr: 0.0010
Epoch 6/100
782/782 [==============================] - 32s 41ms/step - loss: 0.8777 - accuracy: 0.6964 - val_loss: 0.9274 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 7/100
782/782 [==============================]

In [7]:
import tensorflow as tf

checkpoint = tf.train.Checkpoint(model=model)
checkpoint_path = "/content/01.ckpt"  # Specify the path to your checkpoint file, without the extension


In [8]:
# Assuming test_images and test_labels are your test datasets
predictions = model.predict(test_images)

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")


313/313 [==============================] - 1s 4ms/step - loss: 0.4249 - accuracy: 0.8594
Test accuracy: 85.94%
